# <span style="color:red"> Lecture 16 - Merging Data </span>

<font size = "5">

In the previous class we covered ...

- Grouping Data
- Aggregating statistics
- Grouping + Aggregating (11:30am section)

In this class we will cover ...

- Grouping + Aggregating (1:00pm section)
- Merging DataFrames
- Importance of inspecting data before merging

First, let's pick up where we left off from last time...

[Link to Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df_results = pd.read_csv("f1_data_raw/results.csv")

In [ ]:
# Grouping... 
driver_group = df_results.groupby(by = "driverId")
constructor_group = df_results.groupby(by = "constructorId")

driver_mean_points = driver_group["points"].mean()
mean_pts_sorted = driver_mean_points.sort_values(ascending = False)
top_5_drivers = mean_pts_sorted.iloc[:5]

mean_pts_position = driver_group[["points", "positionOrder"]].mean()

top5_one_line = df_results.groupby(by = "driverId")["points"].mean().\
    sort_values(ascending=False).iloc[:5]


In [ ]:
# Aggregating...
pts_col = df_results["points"]

pts_stats = pts_col.agg( ["mean", "std", "min", "max"] )

results_agg = df_results.agg(mean_points = ('points','mean'),
                          sd_points =   ('points','std'),
                          min_points =  ('points','min'),
                          max_points =  ('points','max'))


count_unique = lambda col: len(col.unique())
results_agg = df_results.agg(mean_points = ('points','mean'),
                          mean_laps =   ('laps','mean'),
                          min_points =  ('points','min'),
                          max_points =  ('points','max'),
                          num_drivers = ('driverId', count_unique))

# <span style="color:red"> III. Grouping + Aggregating </span>


<img src="figures/agg.png" alt="drawing" width="400"/>

In [ ]:
drivers_agg = (df_results.groupby("driverId")
                      .agg(mean_points = ('points','mean'),
                           sd_points =   ('points','std'),
                           min_points =  ('points','min'),
                           max_points =  ('points','max'),
                           appearances   = ('points',len)))

print(type(drivers_agg))
print(drivers_agg)

In [ ]:
# driverId is the "Index" column, NOT a regular column
print(drivers_agg.columns.values)

<font size = "4" >

Groupby + Aggregate statistics (multigroup)

Each constructor can have multiple vehicles competing in a given race. We'll group by Race ID and Constructor ID, then aggregate statistics. This will allow us to see how each constructor did relative to the others in a given race.

In [ ]:
teamrace_agg = (df_results.groupby(  ["raceId","constructorId"]    )
                       .agg(mean_points = ('points','mean'),
                            sd_points =   ('points','std'),
                            min_points =  ('points','min'),
                            max_points =  ('points','max'),
                            cars_entered   = ('points',len)))

display(teamrace_agg)
# print(teamrace_agg)

<font size = "4">

Filtering + Grouping + Aggregating: <br>

```python 
.query().groupby().agg()
```

- Another example of "chaining"

In [ ]:
# The following gets a subset of the data using .query()
# In this case we subset the data before computing aggregate statistics
# Note: "filtering" is often the word used to obtain a subset

teamrace_agg500 = (df_results.query("raceId >= 500")
                       .groupby(["raceId","constructorId"])
                        .agg(mean_points = ('points','mean'),
                             sd_points =   ('points','std'),
                             min_points =  ('points','min'),
                             max_points =  ('points','max'),
                             cars_entered   = ('points',len)))
display(teamrace_agg500)

In [ ]:
# maybe we're only interested in the mean
mean_race500 = (df_results.query("raceId >= 500").groupby(["raceId","constructorId"]).
    agg(mean_points = ('points','mean')))

display(mean_race500)
print(type(mean_race500))

<font size = "4">

**Exercise:** Perform the following by chaining. Create a DataFrame where for each race (identified by "raceId") we aggregate the average number of laps and the average number of points.

In [ ]:
# your answer here



<font size = "4">

**Exercise:** Perform the following by chaining. For each constructor (identified by "constructorId"), aggregate the average number of points, then sort in descending order.

- First, begin the chain with the variable ``constructor_group``, we defined above.
- Then, begin the chain with the variable ``df_results``

In [ ]:
# your code here



# <span style="color:red"> Merging </span>


<img src="figures/merge_stats.png" alt="drawing" width="600"/>

<font size = "4">

We have the original DataFrame ("df_results"), and we have aggregate statistics for each driver ("drivers_agg"). We'll merge these two together

In [ ]:
display(df_results.head())

In [ ]:
display(drivers_agg.head())

In [ ]:
# This command merges the "aggregate" information in "driver_agg" into
# "df_results" similar as the figure above.
# The merging variable "on" is determined by "driverId", which is a column
# that is common to both DataFrames
# "how = left" indicates that the left DataFrame is the baseline

results_merge = pd.merge(left = df_results,
                         right = drivers_agg,
                         on = "driverId",
                         how = "left")

In [ ]:
display(results_merge)

<font size = "4">

**Exercise:** Compute a scatter plot of "mean_points" (horizontal axis) vs. "points" (vertical axis). This plot tries to describe how much a driver's performance in individual races deviates from their overall average.

In [ ]:
# your code here



<font size = "4">

**Exercise:** Merge the "teamrace_agg" data into "df_results". This time use the option:

```python
        on = ["raceId","constructorId"]
```

In [ ]:
# your code here



# <span style="color:red"> More on Merging </span>

<font size = "4">

We already loaded in "results.csv". Let's load in "circuits.csv" and "races.csv"

In [ ]:
df_races    = pd.read_csv("f1_data_raw/races.csv")
df_circuits = pd.read_csv("f1_data_raw/circuits.csv")

<font size = "4">

- Multi-file datasets can be visualized with an "Entity Relationship Diagram" (ERD). 
- Depicts how the identifiers in each table are connected.


<img src="figures/erd_f1_simple.png" alt="drawing" width="600"/>

<font size = "4">

- We will try to merge together information on each circuit with the data on individual races.


In [ ]:
print(df_races.columns.values)
print()
print(df_circuits.columns.values)

<font size = "4">

- Both DataFrames have a "name" column. 
- But one refers to the name of the track, the other the name of the race.

In [ ]:
race_names = df_races["name"].sort_values().unique()
circuit_names = df_circuits["name"].sort_values().unique()

In [ ]:
print(race_names)

In [ ]:
print(circuit_names)

<font size = "4">

- If we merge together the names, then "name" will be an ambiguous label.
- Let's make new copies of the DataFrames with less ambiguous column labels


In [ ]:
df_races_new = df_races.rename(columns = {"name" : "race_name"})
df_circuits_new = df_circuits.rename(columns = {"name" : "circuit_name"})

<font size = "5">

Goal: Merge datasets

<img src="figures/merge_goal.png" alt="drawing" width="500"/>


```python
pd.merge(data1,data2,on = ...,how = ...)
```

- Strive to merge only specific columns of data2
- Avoid merging all columns
- Keeping it simple gives you more control over the output

In [ ]:
display(df_races_new[['raceId','circuitId']].head())
print()
display(df_circuits_new[["circuitId","circuit_name"]].head())

In [ ]:
# The "pd.merge()" command combines the information from both datasets
# The first argument is the "primary" datasets
# The second argument is the "secondary" dataset
# The "on" is the common variable that is used for merging
# how = "left" tells Python that the left dataset is the primary one

races_merge = pd.merge(left = df_races_new[['raceId', 'circuitId']],
                       right = df_circuits_new[["circuitId", "circuit_name"]],
                       on = "circuitId",
                       how = "left")

display(races_merge.head())

<font size = "4">

- Suppose I want a DataFrame with the columns:
    - "raceId", "year", "circuitId" from "races.csv"
    - "circuitId", "circuit_name", "location" from "circuits.csv"
- How should I perform the merge?

In [ ]:
races_merge = pd.merge(left = ... ,
                        right = ...,
                        on = ...,
                        how = ...)

<font size = "4">

- Below is another example of merging.
- Here we take "date" column from "races.csv" and merge it with the data from "results.csv"

In [ ]:
# Another example of merging

results_merge = pd.merge(df_results,
                         df_races_new[["raceId","date"]],
                         on = "raceId",
                         how = "left")

display(results_merge.head())

<font size = "5">
<span style="color:red"> Common pitfall: </span> What happens if you don't rename?

In [ ]:
display(df_races.head())
print()
display(df_circuits[["circuitId", "name"]].head())

In [ ]:
# The following code merges the raw data
# which has the "name" column in "races_raw" and "circuits_raw"

races_merge_pitfall = pd.merge(df_races,
                               df_circuits[["circuitId","name"]],
                               on = "circuitId",
                               how = "left")

# Python will internally rename the columns "name_x" (for the left dataset)
# and "name_y" (for the right dataset)

print(races_merge_pitfall.columns.values)
display(races_merge_pitfall.head())

In [ ]:
# Since we made sure to rename the columns in the other DataFrames, this label collision won't happen

races_merge_consistent = pd.merge(df_races_new,
                               df_circuits_new[["circuitId","circuit_name"]],
                               on = "circuitId",
                               how = "left")


print(races_merge_consistent.columns.values)
display(races_merge_consistent.head())

<font size = "4">

**Exercise**: The Pandas ``.merge`` function has a keyword argument called ``suffixes`` which you can use to handle where two columns have the same label.

Use this keyword argument in the cell below to merge the two datasets with conflicting column names. Instead of the default "name_x" and "name_y", make the columns read "name_race" and "name_circuit"

Hint: type ``help(pd.merge)`` and see the examples

In [ ]:
races_merge_choose_suffix = pd.merge(df_races,
                               df_circuits[["circuitId","name"]],
                               on = "circuitId",
                               how = "left",
                               suffixes = ...) # fill in this argument



<font size = "4">

**Exercise**: Merge the columns "lat", "lng", and "alt" from "circuits.csv" into the DataFrame ``races_merge``.

In [ ]:
# Your code here

